# Retrieval and Question Answering

This project will focus on building a retrieve-and-read question answering system for answering questions contained in the [Stanford Question Answering Dataset (SQuAD)](https://rajpurkar.github.io/SQuAD-explorer/). Retrieve-and-read systems typically answer general knowledge queries by (1) retrieving relevant documents, such as Wikipedia pages, that might contain the answer, and (2) using attention-based "reader" models to find the answer in those documents. SQuAD is a reading comprehension task in which models are supposed to answer a question given a paragraph which contains the answer as a subspan. Here's an example from the dataset:

> The further decline of Byzantine state-of-affairs paved the road to a third attack in 1185, when a large Norman army invaded Dyrrachium, owing to the betrayal of high Byzantine officials. Some time later, Dyrrachium—one of the most important naval bases of the Adriatic—fell again to Byzantine hands.
>
> Q: When did the Normans attack Dyrrachium?

The correct answer is `1185`. This is a relatively easy question, since only one date is mentioned in the passage. In its original form, SQuAD does not require a retrieval component; all the information required to answer questions is contained in the corresponding paragraphs. To make this dataset a bit harder, we've shuffled the passages, so given a question you must both (1) find the relevant passage and (2) use it to answer the question. Note that this task is still substantially easier than open-domain question answering over Wikipedia, where models are typically tasked with retrieving millions of paragraphs.

In [2]:
import os
os.environ['CUDA_LAUNCH_BLOCKING'] = "1"

##Setup


In [4]:
!pip install datasets
!pip install transformers

  Using cached datasets-2.1.0-py3-none-any.whl (325 kB)
  Using cached aiohttp-3.8.1-cp37-cp37m-manylinux_2_5_x86_64.manylinux1_x86_64.manylinux_2_12_x86_64.manylinux2010_x86_64.whl (1.1 MB)
  Using cached huggingface_hub-0.5.1-py3-none-any.whl (77 kB)
  Using cached responses-0.18.0-py3-none-any.whl (38 kB)
  Using cached xxhash-3.0.0-cp37-cp37m-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (212 kB)
  Using cached fsspec-2022.3.0-py3-none-any.whl (136 kB)
  Using cached urllib3-1.25.11-py2.py3-none-any.whl (127 kB)
  Using cached frozenlist-1.3.0-cp37-cp37m-manylinux_2_5_x86_64.manylinux1_x86_64.manylinux_2_17_x86_64.manylinux2014_x86_64.whl (144 kB)
  Using cached asynctest-0.13.0-py3-none-any.whl (26 kB)
  Using cached async_timeout-4.0.2-py3-none-any.whl (5.8 kB)
  Using cached aiosignal-1.2.0-py3-none-any.whl (8.2 kB)
  Using cached yarl-1.7.2-cp37-cp37m-manylinux_2_5_x86_64.manylinux1_x86_64.manylinux_2_12_x86_64.manylinux2010_x86_64.whl (271 kB)
  Using cached multidict-6.0.2-c

In [5]:
import datasets
import json
import math
import matplotlib.pyplot as plt
import numpy as np
import pdb
import random
import string
import torch
import torch.nn as nn
import torch.nn.functional as F
import tqdm.notebook
import transformers

from collections import Counter
from multiprocessing import Pool
from nltk.corpus import stopwords
from tqdm import tqdm, trange

## Unsupervised Retrieval: TF-IDF and BM25

We'll use a small dataset of song lyrics to ensure that our retrieval methods are working correctly. We'll begin with TF-IDF and BM25, which are unsupervised methods that compute word overlap between queries and documents. At a high level, these techniques work by figuring out which query words are most useful for discriminating between documents and then ranking documents based on how many times those words appear.

In [6]:
# Load dataset:
with open("lyrics_raw.json", "r") as infile:
    lyrics_dataset = json.load(infile)

# Print some examples:
for song_title, song_lyrics in list(lyrics_dataset.items())[:3]:
    print('\033[1m' + song_title + '\033[0m')
    print(song_lyrics + "\n")

Tim McGraw
He said the way my blue eyes shined Put those Georgia stars to shame that night I said, "That's a lie" Just a boy in a Chevy truck That had a tendency of gettin' stuck On backroads at night And I was right there beside him all summer long And then the time we woke up to find that summer gone But when you think Tim McGraw I hope you think my favorite song The one we danced to all night long The moon like a spotlight on the lake When you think happiness I hope you think that little black dress Think of my head on your chest And my old faded blue jeans When you think Tim McGraw I hope you think of me September saw a month of tears And thankin' God that you weren't here To see me like that But in a box beneath my bed Is a letter that you never read From three summers back It's hard not to find it all a little bittersweet And lookin' back on all of that, it's nice to believe When you think Tim McGraw I hope you think my favorite song The one we danced to all night long The moon l

### Preprocessing Text

We'll begin by preprocessing our data. For now, we just want to convert text into a bag-of-words representation that contains a list of words and counts of how frequently they appear. Given some text, we'll first:
* Remove all punctuation
* Remove line breaks
* Convert all text to lowercase

After that, we'll construct a `Counter()` object which keeps tracks of words and their frequencies. Removing stopwords (i.e., common function words) is another optional step that is commonly used in retrieval settings. Because TF-IDF and BM25 will assign low scores to these words anyway, removing stopwords will have minimal a effect on which documents are retrieved, but it might very slightly speed up inference, depending on your implementation. We've included the NLTK stopword list below; however, please note that you might need to comment out some assertions if you remove stopwords.
 

In [7]:
#nltk_stopwords = set(stopwords.words('english'))
import re

def remove_punctuation(text):
    # Remove punctuation, line breaks, and uppercasing:
    """YOUR CODE HERE"""
    # BEGIN SOLUTION
    text = re.sub(r'[^\w\s]', '', text).lower()
    text = re.sub(r'\n ', '', text)
    # END SOLUTION
    return text
    
def bag_of_words(text):
    # Convert text into a bag-of-words representation using Counter():
    text = remove_punctuation(text)
    """YOUR CODE HERE"""
    # BEGIN SOLUTION
    bag = Counter(text.strip().split(" "))
    return bag
    # END SOLUTION

In [8]:
assert bag_of_words("Ah-ah, ah-ah!") == Counter({'ahah': 2})
assert bag_of_words("helLo heLLo i'm here!") == Counter({'hello': 2, "im": 1, "here": 1})
assert bag_of_words("Testing line break \n line break") == Counter({'line': 2, 'break': 2, 'testing': 1})

### Constructing TF-IDF Matrix

Our first retrieval method is TF-IDF, which assigns a score $\text{tfidf}(t,d)$ to each term $t$ and document $d$. TF-IDF scores consist of (1) term frequency $\text{tf}(t,d)$ in a document and (2) inverse document frequency $\text{idf}(t)$, i.e., the reciprocal of the likelihood that a term will show up in any given document. We often scale these terms by taking the logarithm to ensure that very frequent terms don't dominate the score. Although there are several different formulations of TF-IDF, we'll use the following equations:

\begin{align*}
\text{tf}(t,d) &= \ln(C(t,d) + 1) \\
\text{idf}(t) &= \ln(N/\left|d : C(t,d) > 0\right|) \\
\text{tfidf}(t,d) &= \text{tf}(t,d) \cdot \text{idf}(t)
\end{align*}

where $C(t,d)$ is the number of times term $t$ appears in document $d$, $N$ is the total number of documents, and $\left|d : C(t,d) > 0\right|$ is the number of unique documents containing $t$. We'll discuss how these scores are used for multi-term queries below.

In [9]:
class TFIDF:
    def __init__(self, documents):
        """YOUR CODE HERE"""
        # BEGIN SOLUTION
        self.documents = documents
        self.n = len(documents)
        self.doc_count = []
        self.term_count = {}

        for each in self.documents:
          document = remove_punctuation(each).split(" ")
          self.doc_count.append(Counter(document))

        for i,each in enumerate(self.documents):
          for word in remove_punctuation(each).split(" "):
            if self.term_count.get(word) and self.term_count[word][0]!=i:
              self.term_count[word] = (i,self.term_count[word][1]+1)
            elif not self.term_count.get(word):
              self.term_count[word] = (i,1)
        
        for term in self.term_count.keys():
          self.term_count[term] = self.term_count[term][1]

        # END SOLUTION
        
    
    def __tf(self, term, document_idx):
        """YOUR CODE HERE"""
        # BEGIN SOLUTION
        count = self.doc_count[document_idx]
        tf = count[term]
        return np.log(tf+1)
        # END SOLUTION
    
    def __idf(self, term):
        """YOUR CODE HERE"""
        # BEGIN SOLUTION
        if self.term_count.get(term):
            return np.log(self.n / self.term_count[term])
        else:
            return 0
        # END SOLUTION
    
    def tfidf(self, term, document_idx):
        """YOUR CODE HERE"""
        # BEGIN SOLUTION
        return self.__tf(term, document_idx)* self.__idf(term)
        # END SOLUTION

In [10]:
document_titles = [val[0] for val in lyrics_dataset.items()]
document_to_idx = {val[0]: idx for idx, val in enumerate(lyrics_dataset.items())}
documents = [val[1] for val in lyrics_dataset.items()]
tfidf = TFIDF(documents)

# Some variability in implementation is probably fine here, so you can optionally comment these out:
eps = 0.001
assert abs(tfidf.tfidf("the", document_to_idx["Tim McGraw"])) < eps
assert abs(tfidf.tfidf("blue", document_to_idx["Tim McGraw"]) - 4.014812803551251) < eps
assert abs(tfidf.tfidf("miracle", document_to_idx["Invisible"]) - 6.298343937566328) < eps

### Querying Documents with TF-IDF

Given a multi-term query $q = \{q_0, \ldots, q_k\}$, the simplest way to retrieve documents using TF-IDF is summing over each word:

$$
\text{tfidf}(q,d) = \sum_{i=0}^k \text{tfidf}(q_i, d)
$$

We'll now extend our code to allow multi-term queries and return a ranked list of documents given a query:

In [11]:
class QueryTFIDF(TFIDF):
    def __init__(self, documents):
        super().__init__(documents)
        
    def score_query(self, query, document_idx):
        score = 0
        """YOUR CODE HERE"""
        # 1. Remove punctuation and split query into individual words
        # 2. Call the tfidf() function and sum across words
        # BEGIN SOLUTION
        query = remove_punctuation(query).split(" ")
        for i, word in enumerate(query):
          score += tfidf.tfidf(word, document_idx)
        # END SOLUTION
        return score
        
    def topk(self, query, k=1):
        document_scores = Counter()
        """YOUR CODE HERE"""
        # BEGIN SOLUTION
        for i in range(self.n):
          score = self.score_query(query, i)
          document_scores[i] += score
        # END SOLUTION
        return document_scores.most_common(k)
        
tfidf = QueryTFIDF(documents)
preds = tfidf.topk("we wait on trains", 5)
preds = [(document_titles[pred[0]], pred[1]) for pred in preds]
print(preds)

[('New Romantics', 6.7661824541014), ('How You Get The Girl', 4.9209582839842145), ('Speak Now', 4.224718532231233), ('Innocent', 3.7012080950698527), ('Sad Beautiful Tragic', 2.817697916659227)]


In [12]:
preds = tfidf.topk("pretty girl nightmare", 5)
preds = [(document_titles[pred[0]], pred[1]) for pred in preds]
print(preds)

[('Blank Space', 5.965227287854866), ('How You Get The Girl', 4.867752367233103), ('Girl at Home', 4.100984164844134), ('Ours', 3.965418816079433), ("Mary's Song (Oh My My My)", 3.815061839630246)]


In [13]:
# Helper function for tests:
def predict_song(query):
    document_idx = tfidf.topk(query, 5)[0][0]
    document_title = document_titles[document_idx]
    return document_title

assert predict_song("car") == "Getaway Car"
assert predict_song("we wait on trains") == "New Romantics"
assert predict_song("I wonder if you know") == "I Almost Do"

In [16]:
predict_song("pretty girls")

'Stay Beautiful'

### Improving Retrieval with BM25

BM25 is an alternative retrieval method that fixes some issues with the way that TF-IDF handles very frequent terms and varying document lengths. Once again, there are many different versions of BM25, but we'll be using the following one:

\begin{align*}
\text{df}(t) &= \left|d : C(t,d) > 0\right| \\
\text{idf}(t) &= \ln\left(\frac{N - \text{df}(t) + 0.5}{\text{df}(t) + 0.5} + 1\right) \\
\text{BM25}(q,d) &= \sum_{i=0}^k \text{idf}(q_i,d) \cdot \frac{C(q_i,d) \cdot (k + 1)}{C(q_i,d) + k\cdot \left(1- b + b \cdot N\cdot |d|/\sum_{j}\left|d_j\right|\right)}
\end{align*}

where $k=1.5, b=0.75$ are hyperparameters, $|d|$ is the document length, $N$ is the number of documents, and $\sum_{j}\left|d_j\right|/N$ is the average document length. Note that the inverse document frequency term is different from what we used in TF-IDF.

In [17]:
import time
class BM25:
    def __init__(self, documents, k=1.5, b=0.75):
        self.documents = documents
        self.num_documents = len(documents)
        self.avg_length = 0
        self.doc_count = []
        self.doc_length = []
        self.term_count = {}
        """YOUR CODE HERE"""
        # BEGIN SOLUTION
        for each in self.documents:
          self.avg_length += len(each.split(" "))
          document = remove_punctuation(each).split(" ")
          self.doc_count.append(Counter(document))
          self.doc_length.append(len(document))

        for i,each in enumerate(self.documents):
          for word in remove_punctuation(each).split(" "):
            if self.term_count.get(word) and self.term_count[word][0]!=i:
              self.term_count[word] = (i,self.term_count[word][1]+1)
            elif not self.term_count.get(word):
              self.term_count[word] = (i,1)
        
        for term in self.term_count.keys():
          self.term_count[term] = self.term_count[term][1]

        self.avg_length = self.avg_length/self.num_documents
        # END SOLUTION
    
    def __tf(self, term, document_idx):
        """YOUR CODE HERE"""
        # BEGIN SOLUTION
        document = remove_punctuation(self.documents[document_idx]).split(" ")
        count = Counter(document)
        tf = count[term]
        return np.log(tf+1)
        # END SOLUTION
    
    def __idf(self, term):
        """YOUR CODE HERE"""
        # BEGIN SOLUTION
        if self.term_count.get(term):
          df = self.term_count[term]

          return np.log(((self.num_documents - df + 0.5) / (df + 0.5)) + 1)
        return 0
        # END SOLUTION
        
    def bm25(self, query, document_idx):
        """YOUR CODE HERE"""
        # BEGIN SOLUTION
        score = 0
        query = remove_punctuation(query).split(" ")
        length = self.doc_length[document_idx]
        count = self.doc_count[document_idx]

        for i, word in enumerate(query):
          score += self.__idf(word) * ((count[word] * 2.5) / (count[word] + (1.5 * (0.25 + (0.75 * (length/self.avg_length) ) ))))
        return score
        # END SOLUTION
    
    
    def topk(self, query, k=1):
        document_scores = Counter()
        start = time.time()
        """YOUR CODE HERE"""
        # BEGIN SOLUTION
        for i in range(self.num_documents):
          score = self.bm25(query, i)
          document_scores[i] += score
        # END SOLUTION
        return document_scores.most_common(k)

In [18]:
bm25 = BM25(documents)

def predict_song(query):
    document_idx = bm25.topk(query, 5)[0][0]
    document_title = document_titles[document_idx]
    return document_title

assert predict_song("car") == "Getaway Car"
assert predict_song("we wait on trains") == "New Romantics"
assert predict_song("I wonder if you know") == "I Almost Do"

Because BM25 was designed to handle issues like varying document length and term saturation, it won't have a large effect on our song database, in which most documents have roughly the same length. We'll now switch over to a larger dataset.

## Retrieving Passages from SQuAD

We'll now evaluate our retrieval methods using the Stanford Question Answering Dataset (SQuAD). Because questions are already associated with passages in SQuAD, we'll artificially shuffle the passages to simulate a retrieval setting. In particular, we'll evaluate based on ability to retrieve a gold passage for questions in the validation set of SQuAD. We'll assume access to the following:
* Training set: questions, associated passages, and associated answers
* Validation set: questions, shuffled passages

Because TF-IDF and BM25 are unsupervised retrieval methods, we'll evaluate them directly on the validation set. We'll then work towards supervised neural retrieval and reader models, which will make use of the training set.

In [19]:
from datasets import load_dataset
dataset = load_dataset("squad")

Extracting data files:   0%|          | 0/2 [00:00<?, ?it/s]

Generating train split:   0%|          | 0/87599 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/10570 [00:00<?, ? examples/s]

Dataset squad downloaded and prepared to /root/.cache/huggingface/datasets/squad/plain_text/1.0.0/d6ec3ceb99ca480ce37cdd35555d6cb2511d223b9150cce08a837ef62ffea453. Subsequent calls will reuse this data.


  0%|          | 0/2 [00:00<?, ?it/s]

In [20]:
# Print some example questions and passages from SQuAD
training_data = dataset["train"]

example_idxs = [0,1,9,99]
example_questions = [training_data["question"][idx] for idx in example_idxs]
example_passages = [training_data["context"][idx] for idx in example_idxs]

for idx, question, passage in zip(example_idxs, example_questions, example_passages):
    print('\033[1mQuestion ' + str(idx+1) + ": " + question + '\033[0m')
    print(passage)
    print()

Question 1: To whom did the Virgin Mary allegedly appear in 1858 in Lourdes France?
Architecturally, the school has a Catholic character. Atop the Main Building's gold dome is a golden statue of the Virgin Mary. Immediately in front of the Main Building and facing it, is a copper statue of Christ with arms upraised with the legend "Venite Ad Me Omnes". Next to the Main Building is the Basilica of the Sacred Heart. Immediately behind the basilica is the Grotto, a Marian place of prayer and reflection. It is a replica of the grotto at Lourdes, France where the Virgin Mary reputedly appeared to Saint Bernadette Soubirous in 1858. At the end of the main drive (and in a direct line that connects through 3 statues and the Gold Dome), is a simple, modern stone statue of Mary.

Question 2: What is in front of the Notre Dame Main Building?
Architecturally, the school has a Catholic character. Atop the Main Building's gold dome is a golden statue of the Virgin Mary. Immediately in front of the M

In [21]:
# Do not modify this cell block:
evaluation_data = dataset["validation"]
evaluation_passages = [val["context"] for val in evaluation_data]
evaluation_questions = [val["question"] for val in evaluation_data]

# Shuffle evaluation passages but maintain indices for evaluation:
shuffle_idxs = np.arange(len(evaluation_passages))
random.shuffle(shuffle_idxs)
reverse_idxs = {shuffle_idx : idx for idx, shuffle_idx in enumerate(shuffle_idxs)}
assert reverse_idxs[shuffle_idxs[0]] == 0

evaluation_passages = [evaluation_passages[idx] for idx in shuffle_idxs]
evaluation_questions = [evaluation_questions[idx] for idx in shuffle_idxs]

Because there are typically multiple questions associated with each passage in SQuAD, we'll consider the set of unique passages during retrieval. We'll also write some indexing code to make sure these can be mapped back onto the gold labels:

In [22]:
print("Number of questions: {}".format(len(evaluation_passages)))
print("Number of unique passages: {}".format(len(set(evaluation_passages))))

unique_passages = list(set(evaluation_passages))
passage_to_idx = {passage: idx for idx, passage in enumerate(unique_passages)}
passage_idx_to_unique_idx = [passage_to_idx[passage] for passage in evaluation_passages]

Number of questions: 10570
Number of unique passages: 2067


##TFBM (Do not run)

In [23]:
# It might take a couple minutes to initialize on this larger dataset:
tfidf = QueryTFIDF(unique_passages)
bm25 = BM25(unique_passages)

We typically evaluate the performance of question answering and retrieval systems using a metric known as mean reciprocal rank (MRR):

$$
\text{MRR} = \frac{1}{|Q|}\sum_{i=0}^{|Q|}\frac{1}{\text{rank}_i}
$$

for |Q| queries. Without modifying any of the code in this section, you should expect to see an MRR of 0.7 or above for the TF-IDF model, and 0.8 or above for the BM25 model. Depending on how you implemented TF-IDF and BM25, the following code might take a couple minutes to run. Feel free to reduce `num_examples` during training, and consider revisiting your implementations to improve efficiency if they are too slow. Slight variation from the reported numbers is acceptable and attributable to the randomness induced during shuffling passages.

In [103]:
num_examples = 1000
gold_passages = [passage_idx_to_unique_idx[idx] for idx in range(num_examples)]
real_passages = [list(passage_to_idx.keys())[list(passage_to_idx.values()).index(idx)] for idx in gold_passages]
c=0
e=0
a=[]
mrr_sum = 0.0
exact_match = 0 
for idx, question in tqdm(enumerate(evaluation_questions[:num_examples])):
    rankings = {val[0]: idx for idx, val in enumerate(tfidf.topk(question, 20))}
    gold_passage = gold_passages[idx]
    if idx == 17:
        print("!!!!!!!!!!!!!!!!!!!!")
        if gold_passage in rankings:
          print("Rank: ",rankings[gold_passage])
          print("Q: ", idx, question)
          top = list(rankings.keys())[list(rankings.values()).index(0)]
          ans = list(rankings.keys())[list(rankings.values()).index(rankings[gold_passage])] 
          print("Top: ",list(passage_to_idx.keys())[list(passage_to_idx.values()).index(top)])
          print("Ans: ",list(passage_to_idx.keys())[list(passage_to_idx.values()).index(ans)])
          print("Gold: ",list(passage_to_idx.keys())[list(passage_to_idx.values()).index(gold_passage)])
        else:
          print("No exact match!!!")
          print("Q: ", idx, question)
          top = list(rankings.keys())[list(rankings.values()).index(0)] 
          print("Top: ",list(passage_to_idx.keys())[list(passage_to_idx.values()).index(top)])
          print("Gold: ",list(passage_to_idx.keys())[list(passage_to_idx.values()).index(gold_passage)])
          print("!!!!!!!!!!!!!!!!!!!!")
    rankings = {val[0]: idx for idx, val in enumerate(tfidf.topk(question, 20))}
    #print()
    
    if gold_passage in rankings:
        #print("Gold: ",list(passage_to_idx.keys())[list(passage_to_idx.values()).index(gold_passage)])
        
        if rankings[gold_passage] > 2:
          c+=1
          #print("Count: ",c)
        mrr_sum += 1/(rankings[gold_passage] + 1)
        if rankings[gold_passage] == 0:
            exact_match += 1 
    else:
        e+=1
        a.append(idx)
        #print("No exact match")
        #print("Q: ", idx, question)
        top = list(rankings.keys())[list(rankings.values()).index(0)] 
        #print("Top: ",list(passage_to_idx.keys())[list(passage_to_idx.values()).index(top)])
        #print("Gold: ",list(passage_to_idx.keys())[list(passage_to_idx.values()).index(gold_passage)])

print(c)
print(e)
print(a)
print("TF-IDF MRR: {}".format(mrr_sum / num_examples))
print("TF-IDF Accuracy: {}".format(exact_match / num_examples))




18it [00:03,  5.92it/s]

!!!!!!!!!!!!!!!!!!!!
Rank:  0
Q:  17 What are one of the key cell types of the adaptive immune system?
Top:  Dendritic cells (DC) are phagocytes in tissues that are in contact with the external environment; therefore, they are located mainly in the skin, nose, lungs, stomach, and intestines. They are named for their resemblance to neuronal dendrites, as both have many spine-like projections, but dendritic cells are in no way connected to the nervous system. Dendritic cells serve as a link between the bodily tissues and the innate and adaptive immune systems, as they present antigens to T cells, one of the key cell types of the adaptive immune system.
Ans:  Dendritic cells (DC) are phagocytes in tissues that are in contact with the external environment; therefore, they are located mainly in the skin, nose, lungs, stomach, and intestines. They are named for their resemblance to neuronal dendrites, as both have many spine-like projections, but dendritic cells are in no way connected to th

55it [00:08,  6.49it/s]


KeyboardInterrupt: ignored

In [83]:
t=[45, 104, 128, 158, 191, 254, 279, 292, 296, 302, 305, 306, 309, 315, 329, 330, 331, 352, 358, 360, 361, 366, 375, 382, 427, 434, 471, 473, 484, 492, 498, 514, 527, 552, 568, 571, 584, 594, 596, 634, 644, 646, 670, 689, 695, 697, 749, 758, 789, 799, 805, 812, 863, 901, 904, 928, 995]
b=[121, 158, 191, 279, 282, 292, 296, 302, 305, 309, 315, 329, 331, 358, 361, 366, 367, 427, 434, 473, 484, 492, 514, 527, 552, 568, 594, 596, 634, 644, 646, 670, 689, 695, 752, 758, 785, 789, 799, 812, 863, 904, 928, 995]
d=[0, 13, 16, 17, 25, 30, 33, 34, 35, 44, 54, 79, 80, 100, 101, 102, 109, 110, 121, 123, 146, 148, 150, 158, 160, 161, 163, 167, 184, 189, 200, 209, 221, 227, 230, 233, 235, 256, 279, 284, 286, 291, 300, 303, 305, 306, 307, 311, 313, 315, 325, 328, 333, 336, 343, 345, 352, 357, 358, 365, 368, 369, 375, 378, 384, 387, 395, 396, 405, 413, 424, 426, 431, 434, 461, 464, 466, 470, 494, 497, 502, 504, 508, 512, 514, 515, 534, 539, 562, 577, 584, 604, 629, 644, 646, 647, 651, 664, 665, 666, 672, 680, 689, 695, 715, 716, 717, 728, 732, 738, 740, 742, 752, 757, 759, 765, 766, 771, 785, 794, 812, 815, 819, 841, 845, 848, 862, 864, 878, 882, 902, 922, 955, 962, 963, 967, 968, 971, 985, 995]

(set(d)-((set(b).intersection(set(t)))))

{0,
 13,
 16,
 17,
 25,
 30,
 33,
 34,
 35,
 44,
 54,
 79,
 80,
 100,
 101,
 102,
 109,
 110,
 121,
 123,
 146,
 148,
 150,
 160,
 161,
 163,
 167,
 184,
 189,
 200,
 209,
 221,
 227,
 230,
 233,
 235,
 256,
 284,
 286,
 291,
 300,
 303,
 306,
 307,
 311,
 313,
 325,
 328,
 333,
 336,
 343,
 345,
 352,
 357,
 365,
 368,
 369,
 375,
 378,
 384,
 387,
 395,
 396,
 405,
 413,
 424,
 426,
 431,
 461,
 464,
 466,
 470,
 494,
 497,
 502,
 504,
 508,
 512,
 515,
 534,
 539,
 562,
 577,
 584,
 604,
 629,
 647,
 651,
 664,
 665,
 666,
 672,
 680,
 715,
 716,
 717,
 728,
 732,
 738,
 740,
 742,
 752,
 757,
 759,
 765,
 766,
 771,
 785,
 794,
 815,
 819,
 841,
 845,
 848,
 862,
 864,
 878,
 882,
 902,
 922,
 955,
 962,
 963,
 967,
 968,
 971,
 985}

In [102]:
e=0
a=[]
mrr_sum = 0.0
exact_match = 0
for idx, question in tqdm(enumerate(evaluation_questions[:num_examples])):
    rankings = {val[0]: idx for idx, val in enumerate(bm25.topk(question, 20))}
    gold_passage = gold_passages[idx]
    if idx == 17:
        if gold_passage in rankings:
          print("Rank: ",rankings[gold_passage])
          print("Q: ", idx, question)
          top = list(rankings.keys())[list(rankings.values()).index(0)]
          ans = list(rankings.keys())[list(rankings.values()).index(rankings[gold_passage])] 
          print("Top: ",list(passage_to_idx.keys())[list(passage_to_idx.values()).index(top)])
          print("Ans: ",list(passage_to_idx.keys())[list(passage_to_idx.values()).index(ans)])
          print("Gold: ",list(passage_to_idx.keys())[list(passage_to_idx.values()).index(gold_passage)])
        else:
          print("NOT MATCH")
        break
    if gold_passage in rankings:
        if rankings[gold_passage] > 2:
          print("Count: ",c)
        mrr_sum += 1/(rankings[gold_passage] + 1)
        if rankings[gold_passage] == 0:
            exact_match += 1 
    else:        
        e+=1
        a.append(idx)
        #print("No exact match")
        #print("Q: ", idx, question)
        top = list(rankings.keys())[list(rankings.values()).index(0)] 
        #print("Top: ",list(passage_to_idx.keys())[list(passage_to_idx.values()).index(top)])
        #print("Gold: ",list(passage_to_idx.keys())[list(passage_to_idx.values()).index(gold_passage)])

print(e)
print(a)
print("BM25 MRR: {}".format(mrr_sum / num_examples))
print("BM25 Accuracy: {}".format(exact_match / num_examples))

3it [00:00,  6.09it/s]

Count:  111


8it [00:00, 10.89it/s]

Count:  111


17it [00:01, 10.33it/s]

Rank:  0
Q:  17 What are one of the key cell types of the adaptive immune system?
Top:  Dendritic cells (DC) are phagocytes in tissues that are in contact with the external environment; therefore, they are located mainly in the skin, nose, lungs, stomach, and intestines. They are named for their resemblance to neuronal dendrites, as both have many spine-like projections, but dendritic cells are in no way connected to the nervous system. Dendritic cells serve as a link between the bodily tissues and the innate and adaptive immune systems, as they present antigens to T cells, one of the key cell types of the adaptive immune system.
Ans:  Dendritic cells (DC) are phagocytes in tissues that are in contact with the external environment; therefore, they are located mainly in the skin, nose, lungs, stomach, and intestines. They are named for their resemblance to neuronal dendrites, as both have many spine-like projections, but dendritic cells are in no way connected to the nervous system. Den

The following code will generate a file `bm25_predictions.json` which you will download and submit to the autograder:

In [ ]:
# Please do not modify the following code:
evaluation_data = dataset["validation"]
evaluation_passages = [val["context"] for val in evaluation_data]
evaluation_questions = [val["question"] for val in evaluation_data]
unique_passages = list(dict.fromkeys(evaluation_passages))
passage_to_idx = {passage: idx for idx, passage in enumerate(unique_passages)}
passage_idx_to_unique_idx = [passage_to_idx[passage] for passage in evaluation_passages]
evaluation_bm25 = BM25(unique_passages)
bm25_predictions = []
for idx, question in tqdm(enumerate(evaluation_questions[:1000])):
    rankings = {val[0]: idx for idx, val in enumerate(evaluation_bm25.topk(question, 20))}
    bm25_predictions.append(rankings)
json.dump(bm25_predictions, open("bm25_predictions.json", "w"))

1000it [01:01, 16.36it/s]


### Report: Experimenting with Dense Passage Retrieval

Due to compute constraints, we won't be able to train supervised retrieval models from scratch on Colaboratory/Kaggle notebooks. Instead, we'll load a pretrained DPR model and run some exploratory experiments on it. Without any finetuning, the pretrained DPR model should achieve a MRR of 0.55, compared to 0.8 for the DPR model. Please fill in the code blocks below, experiment with the model a bit, and write a one-page PDF report describing your findings. Consider answering a subset of the following questions, or coming up with your own ideas for the report:

* Do a comparative error analysis of TF-IDF, BM25, and DPR models. What questions does DPR struggle with that TF-IDF and BM25 got right? Are there questions where only DPR can retrieve the correct passage? Provide some examples of each. 
* Can you obtain better performance by ensembling neural and non-neural retrieval models?
* How does the inference speed of DPR compare to your TF-IDF and BM25 models? Can you scale to a larger dataset and/or speed up retrieval by using [FAISS](https://huggingface.co/docs/datasets/faiss_es) or other similarity search methods?
* **Extra credit:** can you improve performance of the DPR model by finetuning it on the SQuAD training data? This is a relatively challenging task, especially in a Colaboratory/Kaggle notebook, and you may need to consult [Karpukhin et al. 2020](https://arxiv.org/pdf/2004.04906.pdf) and the [Huggingface documentation](https://huggingface.co/docs/transformers/training). To construct training examples, you can use either the BM25 model from above, or the pretrained DPR model. 

Make sure you've selected the GPU accelerator for this section:

In [48]:
if torch.cuda.is_available():
    device = torch.device("cuda")
else:
    device = torch.device("cpu")
print("Using device:", device)

Using device: cuda


In [49]:
from transformers import DPRQuestionEncoder, DPRQuestionEncoderTokenizer
from transformers import DPRContextEncoder, DPRContextEncoderTokenizer

context_tokenizer = DPRContextEncoderTokenizer.from_pretrained("facebook/dpr-ctx_encoder-single-nq-base")
context_model = DPRContextEncoder.from_pretrained("facebook/dpr-ctx_encoder-single-nq-base").to(device)
question_tokenizer = DPRQuestionEncoderTokenizer.from_pretrained("facebook/dpr-question_encoder-single-nq-base")
question_model = DPRQuestionEncoder.from_pretrained("facebook/dpr-question_encoder-single-nq-base").to(device)

Downloading:   0%|          | 0.00/28.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/226k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/455k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/492 [00:00<?, ?B/s]

The tokenizer class you load from this checkpoint is not the same type as the class this function is called from. It may result in unexpected tokenization. 
The tokenizer class you load from this checkpoint is 'DPRQuestionEncoderTokenizer'. 
The class this function is called from is 'DPRContextEncoderTokenizer'.


...

Downloading:   0%|          | 0.00/226k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/455k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/493 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/418M [00:00<?, ?B/s]

Some weights of the model checkpoint at facebook/dpr-question_encoder-single-nq-base were not used when initializing DPRQuestionEncoder: ['question_encoder.bert_model.pooler.dense.weight', 'question_encoder.bert_model.pooler.dense.bias']
- This IS expected if you are initializing DPRQuestionEncoder from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing DPRQuestionEncoder from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


Because we're dealing with a relatively small number of documents, we can pre-compute context embeddings and store them in memory. You may want to set `padding=True` and `truncation=True` during tokenization to avoid shape issues with the model:

In [50]:
# Construct context embeddings by passing `unique_passages` into the tokenizer and model.
# Use context_model(ids).pooler_output to get outputs of the correct size.
# This code might take a few seconds to run:
'''YOUR CODE HERE'''
context_embeddings = []
# BEGIN SOLUTION
with torch.no_grad():
  for passage in unique_passages:
    tokens = context_tokenizer(passage, return_tensors="pt", padding=True, truncation=True)["input_ids"].to(device)
    context_embeddings.append(context_model(tokens).pooler_output)
# END SOLUTION
context_embeddings = torch.vstack(context_embeddings)
assert context_embeddings.shape == torch.Size([len(unique_passages), 768])

To obtain scores for each passage in our evaluation set, we'll take the dot product of question embeddings and context embeddings. Hint: you may want to use `torch.topk` because it returns an `indices` value that can easily be used for returning the indices of predicted passages.

In [101]:
num_examples = 1000
gold_passages = [passage_idx_to_unique_idx[idx] for idx in range(num_examples)]

def dpr_topk(question, k=1):
    # Input: a question, without any preprocessing
    # Output: a list of integer-valued indices 
    """YOUR CODE HERE"""
    # BEGIN SOLUTION
    tokens = question_tokenizer(question, return_tensors="pt", padding=True, truncation=True).to(device)
    question_embeddings = question_model(tokens["input_ids"]).pooler_output
    scores =  torch.matmul(question_embeddings, context_embeddings.T)
    indices = [idx.item() for idx in torch.topk(scores.squeeze(), k).indices]
    return indices
    # END SOLUTION
c=0
e=0
mrr_sum = 0.0
exact_match = 0 
a=[]
for idx, question in tqdm(enumerate(evaluation_questions[:num_examples])):
    #print("------")
    #print("Q: ", question)
    rankings = {val: idx for idx, val in enumerate(dpr_topk(question, 20))}
    gold_passage = gold_passages[idx]
    if idx == 17:
        print("!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!")
        #print("Rank: ",rankings[gold_passage])
        print("Q: ", idx, question)
        top = list(rankings.keys())[list(rankings.values()).index(0)]
        #ans = list(rankings.keys())[list(rankings.values()).index(rankings[gold_passage])] 
        print("Top: ",list(passage_to_idx.keys())[list(passage_to_idx.values()).index(top)])
        #print("Ans: ",list(passage_to_idx.keys())[list(passage_to_idx.values()).index(ans)])
        print("Gold: ",list(passage_to_idx.keys())[list(passage_to_idx.values()).index(gold_passage)])
        print("!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!")
    if gold_passage in rankings:
        if rankings[gold_passage] > 2:
          c+=1
        #print("Rank: ",rankings[gold_passage])
        mrr_sum += 1/(rankings[gold_passage] + 1)
        if rankings[gold_passage] == 0:
            exact_match += 1 
    else:
        e+=1
        a.append(idx)
        #print("Q: ", idx, question)
        top = list(rankings.keys())[list(rankings.values()).index(0)] 
        #print("Top: ",list(passage_to_idx.keys())[list(passage_to_idx.values()).index(top)])
        #print("Gold: ",list(passage_to_idx.keys())[list(passage_to_idx.values()).index(gold_passage)])

print("exact:",e)
print("a:",a)
print("DPR MRR: {}".format(mrr_sum / num_examples))
print("DPR Accuracy: {}".format(exact_match / num_examples))

25it [00:00, 44.30it/s]

!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!
Q:  17 What are one of the key cell types of the adaptive immune system?
Top:  The adaptive immune system evolved in early vertebrates and allows for a stronger immune response as well as immunological memory, where each pathogen is "remembered" by a signature antigen. The adaptive immune response is antigen-specific and requires the recognition of specific "non-self" antigens during a process called antigen presentation. Antigen specificity allows for the generation of responses that are tailored to specific pathogens or pathogen-infected cells. The ability to mount these tailored responses is maintained in the body by "memory cells". Should a pathogen infect the body more than once, these specific memory cells are used to quickly eliminate it.
Gold:  Dendritic cells (DC) are phagocytes in tissues that are in contact with the external environment; therefore, they are located mainly in the skin, nose, lungs, stomach, and intestines. They are named for the

490it [00:10, 44.74it/s]


KeyboardInterrupt: ignored

In [ ]:
# Write your experimental code for the report here:
"""YOUR CODE HERE"""
# BEGIN SOLUTION

# END SOLUTION

Note that real-world datasets for open-domain question answering are orders of magnitude larger than the dataset we've been using, often with millions of articles to retrieve from. For example, a standard benchmark task involves open-domain question answering over a subset of English Wikipedia, which is broken down into approximately 21M passages. To learn more about neural retrieval methods, please check out the [DPR paper](https://arxiv.org/abs/2004.04906) or work on [efficient retrieval](https://arxiv.org/pdf/2106.00882.pdf).

## Reader Model

Now that we've built a few models for retrieving relevant passages given a question, we'll build a model that can parse passages in search of correct answers. This part of the assignment is adapted from MIT's 6.864, which is taught by Jacob Andreas, Yoon Kim, and Jim Glass.

We'll build a reading comprehension model that extractively selects spans from passages as possible answers. To get started, we'll write some helper functions for processing data. We'll use the [DistilBERT](https://arxiv.org/abs/1910.01108) model to ensure good performance on Colaboratory/Kaggle notebooks.

In [ ]:
tokenizer = transformers.AutoTokenizer.from_pretrained('distilbert-base-cased')

Below you'll implement the `ans_loc` helper function to determine start and end indices of an answer in a passage. For example, if
- Context = [CLS], protect, yourself, with, an, N, ##9, ##5, mask, [SEP]
- Answer = [CLS], N, ##9, ##5, mask, [SEP]

The the answer location is
- Start postition: 5 (N)
- End postion: 8 (mask)

Hint: you need `ctx_enc['offset_mapping']` to pass all test cases. Refer to this [document](https://huggingface.co/transformers/main_classes/tokenizer.html#transformers.PreTrainedTokenizer.__call__) for information about `offset_mapping`. Briefly speaking, offset_mapping is the character-level position of each token in the input text. for the `i`-th token in a input sequence,

In [ ]:
# Given a passage 'ctx' and an answer 'ans', return the start and end indices of the answer:
def ans_loc(ctx, ans, verbose=False):
    start_loc = -1
    end_loc = -1
    ctx_enc = tokenizer(ctx, return_offsets_mapping=True, verbose=False)
    ans_enc = tokenizer(ans, return_offsets_mapping=True)

    """YOUR CODE HERE"""
    # BEGIN SOLUTION

    ctx_str = (" ".join(ctx_enc.tokens()[1:-1])).replace("#",'').replace(" ",'')
    ans_str = (" ".join(ans_enc.tokens()[1:-1])).replace("#",'').replace(" ",'')

    start_chr = ctx_str.find(ans_str)
    end_chr = start_chr + len(ans_str) - 1

    new_offset = ctx_enc["offset_mapping"]
    
    for i in range(len(new_offset)):
      if i == len(new_offset) - 1:
        continue
      if new_offset[i][1] != new_offset[i+1][0]:
        new_offset[i+1] = (new_offset[i+1][0] - new_offset[i+1][0] + new_offset[i][1], new_offset[i+1][1] - new_offset[i+1][0] + new_offset[i][1])
    
    for i,index in enumerate(new_offset):
      if (index[0] >= start_chr or index[1] > start_chr) and start_loc == -1:
        start_loc = i
      if index[1] > end_chr and end_loc == -1:
        end_loc = i
    # END SOLUTION

    return start_loc, end_loc

you can check for substrings at
 the character level and then use offset_mapping to project back onto token IDs.

In [ ]:
# -------------- Test Case 1 ----------------- #

print('------------- Test Case 1 -------------')
ctx_c1 = 'You can protect yourself by wearing an N95 mask.'
ans_c1 = 'wearing an N95 mask'

start_loc, end_loc = ans_loc(ctx_c1, ans_c1, verbose=True)
print(f'The start location is {start_loc}, and the end location is {end_loc}')

if (start_loc, end_loc) == (6, 11):
    print('\nYour implementation is correct for case 1')
else:
    print('\nYour implementation failed on case 1')

# -------------- Test Case 2 ----------------- #

print('\n------------- Test Case 2 -------------')
ctx_c2 = 'split with Luckett and Roberson'
ans_c2 = 'Luckett and Rober'

start_loc, end_loc = ans_loc(ctx_c2, ans_c2, verbose=True)
print(f'The start location is {start_loc}, and the end location is {end_loc}')
if (start_loc, end_loc) == (3, 7):
    print('\nYour implementation is correct for case 2')
else:
    print('\nYour implementation failed on case 2')

# -------------- Test Case 3 ----------------- #

print('\n------------- Test Case 3 -------------')
ctx_c2 = 'The UK government has spent £250 million in the construction of the island'
ans_c2 = '250 million'

start_loc, end_loc = ans_loc(ctx_c2, ans_c2, verbose=True)
print(f'The start location is {start_loc}, and the end location is {end_loc}')
if (start_loc, end_loc) == (6, 8):
    print('\nYour implementation is correct for case 3')
else:
    print('\nYour implementation failed on case 3')

------------- Test Case 1 -------------
The start location is 6, and the end location is 11

Your implementation is correct for case 1

------------- Test Case 2 -------------
The start location is 3, and the end location is 7

Your implementation is correct for case 2

------------- Test Case 3 -------------
The start location is 6, and the end location is 8

Your implementation is correct for case 3


We'll preprocess the dataset using the `ans_loc` function from above to determine start and end indices for each question in our training set. Because this function might take a few minutes to run, we'll save our preprocessed data to a file which can be saved locally:

In [ ]:
def proc_line_init(tokenizer_for_squad):
    global tokenizer
    tokenizer = tokenizer_for_squad

# Preprocess one SQuAD data point
def proc_line(sq):
    ctx = sq['context']
    ans = sq['answers']['text'][0]

    ctx_ids = tokenizer.encode(ctx, verbose=False)
    ans_ids = tokenizer.encode(ans)

    if len(ctx_ids) > 448:
        return None

    start_pos = None
    end_pos = None
    
    start_pos, end_pos = ans_loc(ctx, ans)
    
    sq['start_position'] = start_pos
    sq['end_position'] = end_pos
    return sq

# Preprocess SQuAD corpus with tqdm multithreading
def preproc(squad_list, threads, tokenizer):

    with Pool(threads, initializer=proc_line_init, initargs=(tokenizer,)) as p:
        squad_proc = list(tqdm(p.imap(proc_line, squad_list), total=len(squad_list)))

    squad_proc = [x for x in squad_proc if x]
    return squad_proc

# Feel free to comment out this code and load squad_proc from a file 
squad_list = [x for x in dataset['train']]
squad_proc = preproc(squad_list, 16, tokenizer)

100%|██████████| 87599/87599 [00:46<00:00, 1874.30it/s]


We recommend downloading `preprocessed_squad.json` in case the session dies:

In [ ]:
#json.dump(squad_proc, open("preprocessed_squad.json", 'w'))
squad_proc = json.load(open("preprocessed_squad.json", "r"))

In [ ]:
# Test if your preprocessing is correct
def test_preproc(sq, tokenizer):
    gt_ans = sq['answers']['text'][0]
    start_pos = sq['start_position']
    end_pos = sq['end_position']

    tok_outputs = tokenizer(sq['context'], return_offsets_mapping=True)
    ctx_ids = tok_outputs['input_ids']
    offsets = tok_outputs['offset_mapping']

    start_pos_char = offsets[start_pos][0]
    end_pos_char = offsets[end_pos][1]

    pred_ans = sq['context'][start_pos_char: end_pos_char]
    
    
    #print(f'The annotated answer: {gt_ans}')
    #print(f'The predicted answer: {pred_ans}')
    if gt_ans == pred_ans:
        return True
    else:
        print("-------------------------------")
        print(f'The annotated answer: {gt_ans}')
        print(f'The predicted answer: {pred_ans}')
        return False

c=0
for each in (squad_proc):
  
  if test_preproc(each, tokenizer):
    c+=1
c/len(squad_proc)

-------------------------------
The annotated answer: 8th
The predicted answer: 18th
-------------------------------
The annotated answer: Fifty-seven
The predicted answer: . Fifty-seven
-------------------------------
The annotated answer: Catholic research university
The predicted answer: AYM) is a Catholic research university
-------------------------------
The annotated answer: Our Lady of the Lake
The predicted answer: Lac means "Our Lady of
-------------------------------
The annotated answer: the Virgin Mary
The predicted answer: patron saint, the Virgin
-------------------------------
The annotated answer: 1,250
The predicted answer: campus covers
-------------------------------
The annotated answer: in the late 1990s
The predicted answer: to fame in the late
-------------------------------
The annotated answer: singing and dancing
The predicted answer: various singing and dancing
-------------------------------
The annotated answer: 2003
The predicted answer: Love
------------

0.9839980756454606

## DistilBERT

We'll now define our reader model. Our model consists of a DistilBERT language model, followed by a dropout layer, followed by a linear projection onto logits over possible start and end positions. Below, you'll define the start and end logits by making use of `self.qa_outputs` and compute `total_loss` by comparing these logits with `start_positions` and `end_positions`.

In [ ]:
import torch.nn as nn

class ModelOutputs:
    def __init__(self, start_logits=None, end_logits=None, loss=None):
        self.start_logits = start_logits
        self.end_logits = end_logits
        self.loss = loss

class QuestionAnsweringModel(nn.Module):

    def __init__(self, lm=None, dropout=0.2):
        '''
        lm:         a pretrained transformer language model
        dropout:    dropoutrate for the dropout layer
        '''
        super(QuestionAnsweringModel, self).__init__()
        self.qa_outputs = nn.Linear(lm.config.dim, 2)
        self.lm = lm
        self.dropout = nn.Dropout(dropout)
    
    def forward(self, input_ids=None, attention_mask=None,
                start_positions=None, end_positions=None):
        '''
        input_ids:          ids of the concatenated input tokens
        attention_mask:     concatenated attention masks (ques+ctx)
        start_positions:    labels of the start positions of the answers
        end_positions:      labels of the end positions of the answers
        '''
        
        lm_output = self.lm(
            input_ids,
            attention_mask
        )

        hidden_states = lm_output.last_hidden_state
        hidden_states = self.dropout(hidden_states)

        start_logits = None
        end_logits = None

        """YOUR CODE HERE"""
        # Pass hidden states into a linear layer to get start and end logits:
        # start_logits.size() should be [batch_size, seq_len]
        # end_logits.size() should also be [batch_size, seq_len]
        # BEGIN SOLUTION
        out = self.qa_outputs(hidden_states)
        start_logits = out[:,:,0].squeeze()
        end_logits = out[:,:,1].squeeze()
        # END SOLUTION

        total_loss = None
        if start_positions is not None and end_positions is not None:

            loss_fct = nn.CrossEntropyLoss()
            total_loss = 0.0
            """YOUR CODE HERE"""
            # 1. calculate start_losses with
            #       a. start_logits
            #       b. start_positions
            #
            # 2. calculate end_losses with end_logits
            #       a. end_logits
            #       b. end_positions
            #
            # 3. set total loss equal to the average of both
            # BEGIN SOLUTION
            start_losses = loss_fct(start_logits, start_positions)
            end_losses = loss_fct(end_logits, end_positions)
            total_loss = torch.add(start_losses,end_losses)/2
            # END SOLUTION
            
        return ModelOutputs(
            start_logits=start_logits,
            end_logits=end_logits,
            loss=total_loss)

In [ ]:
# Initialize the QA model and use GPU
lm_pretrained = transformers.AutoModel.from_pretrained('distilbert-base-cased')
model = QuestionAnsweringModel(lm_pretrained)
model = model.to('cuda')
device = 'cuda'

Downloading:   0%|          | 0.00/251M [00:00<?, ?B/s]

Some weights of the model checkpoint at distilbert-base-cased were not used when initializing DistilBertModel: ['vocab_projector.bias', 'vocab_transform.weight', 'vocab_projector.weight', 'vocab_transform.bias', 'vocab_layer_norm.weight', 'vocab_layer_norm.bias']
- This IS expected if you are initializing DistilBertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing DistilBertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [ ]:
# Hyper-parameters: you could try playing with different settings
num_epochs = 1
learning_rate = 3e-5
weight_decay = 1e-5
eps = 1e-6
batch_size = 32
warmup_rate = 0.05
ques_max_length = 64
ctx_max_length = 448

# Calculating the number of warmup steps
num_training_cases = len(squad_proc)
t_total = (num_training_cases // batch_size + 1) * num_epochs
ext_warmup_steps = int(warmup_rate * t_total)

# Initializing an AdamW optimizer
ext_optim = torch.optim.AdamW(model.parameters(), lr=learning_rate,
                              eps=eps, weight_decay=weight_decay)

# Initializing the learning rate scheduler [details are in the BERT paper]
ext_sche = transformers.get_linear_schedule_with_warmup(
    ext_optim, num_warmup_steps=ext_warmup_steps, num_training_steps=t_total
)

print("***** Training Info *****")
print("  Num examples = %d" % t_total)
print("  Num Epochs = %d" % num_epochs)
print("  Batch size = %d" % batch_size)
print("  Total optimization steps = %d" % t_total)

***** Training Info *****
  Num examples = 2729
  Num Epochs = 1
  Batch size = 32
  Total optimization steps = 2729


We provide some helper functions for preprocessing:

In [ ]:

def gather_batch(batch):
    ctx_batch  = [x['context'] for x in batch]
    ques_batch = [x['question'] for x in batch]
    ans_batch  = [x['answers']['text'][0] for x in batch]

    start_positions = [x['start_position'] for x in batch]
    end_positions = [x['end_position'] for x in batch]

    return ctx_batch, ques_batch, ans_batch, start_positions, end_positions

Note that we'll concatenate questions and answers before feeding them into the reader model, so that inputs look like:
> `[CLS] <question ids> [SEP] <context ids> [SEP]`

Even though you aren't required to modify anything in the vectorizing code, you'll need to handle the indexing to recover predicted spans during inference. We also provide some code for handling of attention masks.

In [ ]:
def vectorize_batch(batch, tokenizer):
    ctx_batch, ques_batch, ans_batch, start_positions, end_positions = gather_batch(batch)

    # Encode the context passage
    ctx_encode = tokenizer.batch_encode_plus(
        ctx_batch,
        max_length = ctx_max_length,
        truncation = True,
        padding = 'longest',
        return_attention_mask = True,
        return_tensors = 'pt'
    )

    # Encode the questions
    ques_encode = tokenizer.batch_encode_plus(
        ques_batch,
        max_length = ques_max_length,
        truncation = True,
        padding = 'longest',
        return_attention_mask = True,
        return_tensors = 'pt'
    )

    # Get the actual sequence lengths of question tensors
    ques_seq_len = ques_encode['input_ids'].size(1)

    # Move the training batch to GPU
    ctx_ids        = ctx_encode['input_ids'].cuda()
    ctx_attn_mask  = ctx_encode['attention_mask'].cuda()
    ques_ids       = ques_encode['input_ids'].cuda()
    ques_attn_mask = ques_encode['attention_mask'].cuda()

    # Remove the [CLS] token of the contexts IDs before concatenation
    ctx_ids = ctx_ids[:, 1:]
    ctx_attn_mask = ctx_attn_mask[:, 1:]

    # Concatenate questions and contexts
    input_ids = torch.cat([ques_ids, ctx_ids], dim=1)
    input_attn_mask = torch.cat([ques_attn_mask, ctx_attn_mask], dim=1)

    # Move start and end positions to the GPU
    start_positions = torch.LongTensor(start_positions).cuda()
    end_positions   = torch.LongTensor(end_positions).cuda()

    # update the start_positions and end_positions variables accordingly
    # This is necessary for the following reasons
    # 
    # 1. We concatenated the questions and contexts
    # 2. We removed the [CLS] token (the first token) of the contexts

    start_positions += ques_seq_len - 1
    end_positions += ques_seq_len - 1

    return input_ids, input_attn_mask, start_positions, end_positions

##Train


In [ ]:
model.train()
max_grad_norm = 1

'''
The training can take up to an hour (~30min in average)
Consider using less training data to validate your implementation
'''

#squad_proc = squad_proc[:10000]
num_training_cases = len(squad_proc)

step_id = 0
for _ in range(num_epochs):

    random.shuffle(squad_proc)

    for i in range(0, num_training_cases, batch_size):
        batch = squad_proc[i: i + batch_size]
        input_ids, input_attn_mask, start_positions, end_positions = vectorize_batch(batch, tokenizer)

        model.zero_grad() # Does the same as ext_optim.zero_grad()
        
        # Get the model outputs, including (start, end) logits and losses
        # stored as a ModelOutput object
        outputs = model(            
            input_ids, input_attn_mask, start_positions, end_positions
        )

        # Back-propagate the loss signal and clip the gradients
        loss = outputs.loss.mean()
        loss.backward()
        torch.nn.utils.clip_grad_norm_(model.parameters(), max_grad_norm)

        # Update neural network parameters and the learning rate
        ext_optim.step()
        ext_sche.step() # Update learning rate for better convergence


        if step_id % 100 == 0:
            print(f'At step {step_id}, the extraction loss = {loss}')
        
        step_id += 1

print('Finished Training')

At step 0, the extraction loss = 5.650728225708008
At step 100, the extraction loss = 4.11710262298584
At step 200, the extraction loss = 3.4899940490722656
At step 300, the extraction loss = 2.940316677093506
At step 400, the extraction loss = 2.0152781009674072
At step 500, the extraction loss = 2.666785717010498
At step 600, the extraction loss = 1.6730377674102783
At step 700, the extraction loss = 1.6442193984985352
At step 800, the extraction loss = 2.194803237915039
At step 900, the extraction loss = 1.5486451387405396
At step 1000, the extraction loss = 1.4193776845932007
At step 1100, the extraction loss = 1.3721040487289429
At step 1200, the extraction loss = 0.981627881526947
At step 1300, the extraction loss = 1.5871152877807617
At step 1400, the extraction loss = 1.591651439666748
At step 1500, the extraction loss = 1.0827834606170654
At step 1600, the extraction loss = 2.038727283477783
At step 1700, the extraction loss = 1.279815435409546
At step 1800, the extraction los

With our reference implementation, training loss is equal to:
* `At step 0: 6.18`
* `At step 100: 4.01`
* `At step 200: 2.63`
* `At step 1000: 1.65`
* `At step 2700: 1.24`

We'll next move on to writing some evaluation code for our reader model. Standard evaluation metrics for extractive, or span-based QA models are exact match (EM) and F1 scores.
- EM: how many predicted answers are exactly the same as the annotated answers
- F1: how many words in the predicted answers overlap the annotated answers.

In other words, the calculations of EM and F1 scores are:
- EM = num_same_answer / num_all_questions
- F1 = (2 * precision * recall) / (precision + recall), where
    - precision = num_overlap_words / num_predicted_answer_words
    - recall = num_overlap_words / num_annotated_answer_words

In [ ]:
def ans_pair_metric(a_pred, a_gt, tokenizer):
    '''
    a_pred: the predicted answer text
    a_gt: the groundtruth answer text
    '''
    
    # Exclude the special tokens
    pred_ids = tokenizer.encode(a_pred)[1: -1]
    gt_ids = tokenizer.encode(a_gt)[1: -1]
    
    len_pred = len(pred_ids)
    len_gt = len(gt_ids)
    num_same = 0
    for word_id in pred_ids:
        if word_id in gt_ids:
            num_same += 1.
    
    em = float(a_pred == a_gt)
    if num_same == 0:
        f1 = 0
    else:
        prec = num_same / len_pred
        recall = num_same / len_gt
        f1 = 2 * prec * recall / (prec + recall)

    return em, f1

We have to modify the evaluation metrics a bit because many questions in SQuAD have multiple ground truth answers:

In [ ]:
def one_to_many_metric(a_pred, a_gt_list, tokenizer):
    '''
    a_pred: the predicted answer text
    a_gt_list: the provided ground-truth answer list
    '''
    metric = np.array([ans_pair_metric(a_pred, x, tokenizer) for x in a_gt_list])

    em = metric[:, 0]
    f1 = metric[:, 1]
    return em.max(), f1.max()

Finally, we'll write some code to convert logits into indices of possible answers. `logits_to_ans_loc` is a decoding function that converts predicted start and end logits to an actual answer span $(i,j)$, where
- word $w_i$ has a high `start_logit`
- word $w_j$ has a high `end_logit`.

Inputs and outputs
- inputs: `start_logits` and `end_logits`
- outputs
    - st_loc: the index of the start token of the predicted answer
    - ed_loc: the index of the end token of the predicted answer

Note:
- We don't consider answers more than 30 tokens
- Make sure `ed_loc >= st_loc`
- Higher start/end logits stands for higher probability that a word could be the start/end point of an answer

Please implement three strategies for getting answer span $(i,j)$
- `greedy_left_to_right` - Select $i = \text{argmax}_i S_{start}^i$, then select $j = argmax_j S_{end}^j$. ($i \le j$)
- `greedy_right_to_lett` - Select $j = argmax_j S_{end}^j$, then select $i = argmax_i \ S_{start}^i$ ($i \le j$)
- `joint` - Select $(i, j)$ by $i, j = argmax_{i, j} S_{start}^i + S_{end}^j (i \le j)$

s=[1,2,3,4,5,6,2,3,4]
e=[4,5,6,2,3,5,6,1,2]

In [ ]:
def logits_to_ans_loc(start_logits, end_logits, mode='joint'):
    '''
    Input sizes -
        start_logits.size() = [batch_size, seq_len]
        end_logits.size() = [batch_size, seq_len]

    Output sizes -
        st_loc.size() = (batch_size,)
        ed_loc.size() = (batch_size,)
    '''
    bs, seq_len = start_logits.size()

    st_loc = None
    ed_loc = None
    
    # Find the span (i, j) that could be an answer
    # to the question, based on the predicted
    # start_logits and end_logits with three modes
    #   - greedy_left_to_right
    #   - greedy_right_to_left
    #   - joint
    #pos_idx = torch.range(0, seq_len - 1).cuda().unsqueeze(0)
    
    if mode == 'greedy_left_to_right':
        """YOUR CODE HERE"""
        # BEGIN SOLUTION
        st_loc = torch.argmax(start_logits, dim=-1)
        mask = torch.zeros_like(end_logits)

        #mask = mask.scatter(1, (st_loc-1).unsqueeze(1), -1e+10)
        #mask = torch.flip(torch.cumsum(torch.flip(mask,[1]),-1),[1])
        #end_masked = torch.add(end_logits, mask)
        #ed_loc = torch.argmax(end_masked, dim=-1)
        
        #or
        for row, value in enumerate(st_loc):
          mask[row, 0:value] = -1e+10
        end_masked = mask + end_logits
        ed_loc = torch.argmax(end_masked,dim=-1)

        # END SOLUTION

    if mode == 'greedy_right_to_left':
        """YOUR CODE HERE"""
        # BEGIN SOLUTION
        #ed_loc = torch.argmax(end_logits, dim=-1)
        
        #mask = torch.zeros_like(start_logits)
        #mask = mask.scatter(1, (ed_loc-1).unsqueeze(1), -1e+10)
        #mask = torch.cumsum(mask,-1)
        #st_masked = torch.add(start_logits, mask)
        
        #st_loc = torch.argmax(st_masked, dim=-1)

        ed_loc = torch.argmax(end_logits, dim=-1)
        mask = torch.zeros_like(start_logits)

        for row, value in enumerate(ed_loc):
          mask[row, value:] = -1e+10
        st_masked = mask + start_logits
        st_loc = torch.argmax(st_masked,dim=-1)
 
        # END SOLUTION

    if mode == 'joint':
        """YOUR CODE HERE"""
        # Note: vectorizing this implementation can be a bit tricky, so we'll set our grade thresholds
        #       based on the greedy solutions. Our reference solution uses an outer sum, followed by
        #       torch.tril and torch.repeat to mask out cases where start_loc > end_loc. We then flatten 
        #       and compute and argmax before using torch.div and torch.remainder to index into it.
        # BEGIN SOLUTION
        st_loc = torch.LongTensor(start_logits.shape[0])
        ed_loc = torch.LongTensor(end_logits.shape[0])
        for n in range(start_logits.shape[0]):
          start = start_logits[n]
          end = end_logits[n]
          max = -1e+10
          maxi = 0
          maxj = 0
          for i, start_val in enumerate(start):
            for j in range(len(end)-i):
              if start[i]+end[i+j] > max:
                max = start[i]+end[i+j]
                maxi = i
                maxj = i+j
          st_loc[n] = maxi
          ed_loc[n] = maxj
        # END SOLUTION
        
        
    return st_loc, ed_loc

In [ ]:
# Prepare the dev set of SQuAD for evaluation
dev_set = [x for x in dataset['validation']]
num_dev_cases = len(dev_set)

eval_batch_size = 64

# `ans_pred_list` stores the predicted answers
# in the same order as the contexts of the dev set
ans_pred_list_ltr = []
ans_pred_list_rtl = []
ans_pred_list_joint = []
ans_gt_list = [x['answers']['text'] for x in dev_set]

In [ ]:
model.eval()
print(num_dev_cases)
for i in range(0, num_dev_cases, eval_batch_size):
    eval_batch = dev_set[i: i + eval_batch_size]
    ques = [x['question'] for x in eval_batch]
    ctx  = [x['context'] for x in eval_batch]

    # Encode the contexts
    ctx_encode = tokenizer.batch_encode_plus(
        ctx,
        max_length = ctx_max_length,
        truncation = True,
        padding = 'longest',
        return_attention_mask = True,
        return_tensors = 'pt',
        return_offsets_mapping = True
    )

    # Encode the questions
    ques_encode = tokenizer.batch_encode_plus(
        ques,
        max_length = ques_max_length,
        truncation = True,
        padding = 'longest',
        return_attention_mask = True,
        return_tensors = 'pt'
    )

    # get the actual question sequence lengths
    ques_len = ques_encode['input_ids'].size(1)

    """YOUR CODE HERE"""
    # Concatenate the input ids and attention masks
    # of questions and contexts. Refer to the training
    # loop for implementation hints
    # BEGIN SOLUTION

    input_ids = torch.cat([ques_encode['input_ids'], ctx_encode['input_ids']],dim = 1).cuda()
    input_attn_mask = torch.cat([ques_encode['attention_mask'], ctx_encode['attention_mask']],dim = 1).cuda()

    # END SOLUTION

    with torch.no_grad():
        outputs = model(
            input_ids,
            attention_mask = input_attn_mask,
        )
    
    """YOUR CODE HERE"""
    # Obtain the predicted start and end logits
    # We drop the start and end logits of question tokens
    # and only keep the logits of
    #
    #       [SEP] or [PAD], ctx_1, ctx_2, ..., [SEP]
    #
    # keep the first special token, which is the tail
    # or padding token of the tokenized question, to
    # help you do later decoding more easily 
    # BEGIN SOLUTION
    start_logits_pred = outputs.start_logits[:,ques_len:]
    end_logits_pred =  outputs.end_logits[:,ques_len:]
    # END SOLUTION

    st_locs_ltr, ed_locs_ltr = logits_to_ans_loc(
        start_logits_pred, end_logits_pred, mode='greedy_left_to_right'
    )

    st_locs_rtl, ed_locs_rtl = logits_to_ans_loc(
        start_logits_pred, end_logits_pred, mode='greedy_right_to_left'
    )
    
    #st_locs_joint, ed_locs_joint = logits_to_ans_loc(start_logits_pred, end_logits_pred, mode='joint')

    num_pred_answer = st_locs_ltr.size(0)

    # ---------------------------------------------- #
    #
    # Store predicted answer texts in `ans_pred_list`
    # 1. `ans_pred_list` should look like
    #             ['ans_txt_1', 'ans_txt_2', ....]
    #
    # 2. `len(ans_pred_list)` should equal to
    #             `len(dev_set)`
    #
    # ---------------------------------------------- #

    for j in range(num_pred_answer):

        st_char_ltr = ctx_encode['offset_mapping'][j][st_locs_ltr[j]][0]
        ed_char_ltr = ctx_encode['offset_mapping'][j][ed_locs_ltr[j]][1]
        ans_pred_list_ltr.append(ctx[j][st_char_ltr: ed_char_ltr])

        st_char_rtl = ctx_encode['offset_mapping'][j][st_locs_rtl[j]][0]
        ed_char_rtl = ctx_encode['offset_mapping'][j][ed_locs_rtl[j]][1]
        ans_pred_list_rtl.append(ctx[j][st_char_rtl: ed_char_rtl])

        #st_char_joint = ctx_encode['offset_mapping'][j][st_locs_joint[j]][0]
        #ed_char_joint = ctx_encode['offset_mapping'][j][ed_locs_joint[j]][1]
        #ans_pred_list_joint.append(ctx[j][st_char_joint: ed_char_joint])
    
    # print(ans_pred_list_ltr)
    
    # ---------------------------------------------- #

# Print the evaluation results
#
# The performance is decided by both training quality
# AND the implementation of the `logits_to_anc_loc` function

# Target result
# - EM: 68.6%
# - F1: 77.7%

print('Evaluating greedy_left_to_right strategy')
metric = np.array([one_to_many_metric(x, y, tokenizer) for x, y in zip(ans_pred_list_ltr, ans_gt_list)])
em = metric[:, 0].mean()
f1 = metric[:, 1].mean()

print(f'EM = {em}')
print(f'F1 = {f1}')

# Target result
# - EM: 67.97%
# - F1: 80.89%

print('\nEvaluating greedy_right_to_left strategy')
metric = np.array([one_to_many_metric(x, y, tokenizer) for x, y in zip(ans_pred_list_rtl, ans_gt_list)])
em = metric[:, 0].mean()
f1 = metric[:, 1].mean()

print(f'EM = {em}')
print(f'F1 = {f1}')

# Target result
# - EM: 68.99%
# - F1: 81.77%

"""print('\nEvaluating joint strategy')
metric = np.array([one_to_many_metric(x, y, tokenizer) for x, y in zip(ans_pred_list_joint, ans_gt_list)])
em = metric[:, 0].mean()
f1 = metric[:, 1].mean()

print(f'EM = {em}')
print(f'F1 = {f1}')"""

10570
Evaluating greedy_left_to_right strategy
EM = 0.6557237464522233
F1 = 0.785831225483898

Evaluating greedy_right_to_left strategy
EM = 0.5301797540208136
F1 = 0.7132070064804974


"print('\nEvaluating joint strategy')\nmetric = np.array([one_to_many_metric(x, y, tokenizer) for x, y in zip(ans_pred_list_joint, ans_gt_list)])\nem = metric[:, 0].mean()\nf1 = metric[:, 1].mean()\n\nprint(f'EM = {em}')\nprint(f'F1 = {f1}')"

## Putting It All Together: Read-and-Retrieve

Now that we've implemented both reader and retrieval models, we'll combine them into a complete open-domain questioning system.

In [ ]:
evaluation_passages = [val["context"] for val in evaluation_data]
evaluation_questions = [val["question"] for val in evaluation_data]

num_examples = 1000
modified_dev_set = []
for idx, question in tqdm(enumerate(evaluation_questions[:num_examples])):
    passage = unique_passages[evaluation_bm25.topk(question)[0][0]]
    modified_dev_set.append({"question": question, "context": passage})
num_dev_cases = len(modified_dev_set)

1000it [01:01, 16.34it/s]


In [ ]:
print("Number of evaluation examples: {}".format(num_dev_cases))

evaluation_data = dataset["validation"]
dev_set = [x for x in dataset['validation']][:num_dev_cases]
ans_gt_list = [val["answers"]["text"] for val in dev_set]

ans_pred_list_ltr = []
for i in range(0, num_dev_cases, eval_batch_size):
    eval_batch = modified_dev_set[i: i + eval_batch_size]
    ques = [x['question'] for x in eval_batch]
    ctx  = [x['context'] for x in eval_batch]

    # Encode the contexts
    ctx_encode = tokenizer.batch_encode_plus(
        ctx,
        max_length = ctx_max_length,
        truncation = True,
        padding = 'longest',
        return_attention_mask = True,
        return_tensors = 'pt',
        return_offsets_mapping = True
    )

    # Encode the questions
    ques_encode = tokenizer.batch_encode_plus(
        ques,
        max_length = ques_max_length,
        truncation = True,
        padding = 'longest',
        return_attention_mask = True,
        return_tensors = 'pt'
    )

    # get the actual question sequence lengths
    ques_len = ques_encode['input_ids'].size(1)

    input_ids = torch.cat(
        [ques_encode['input_ids'], ctx_encode['input_ids']],
        dim = 1
    ).cuda()

    input_attn_mask = torch.cat(
        [ques_encode['attention_mask'], ctx_encode['attention_mask']],
        dim = 1
    ).cuda()
    with torch.no_grad():
        outputs = model(
            input_ids,
            attention_mask = input_attn_mask,
        )

    start_logits_pred = outputs.start_logits[:, ques_len:]
    end_logits_pred = outputs.end_logits[:, ques_len:]
    st_locs_ltr, ed_locs_ltr = logits_to_ans_loc(
        start_logits_pred, end_logits_pred, mode='greedy_left_to_right'
    )
    num_pred_answer = st_locs_ltr.size(0)
    for j in range(num_pred_answer):
        st_char_ltr = ctx_encode['offset_mapping'][j][st_locs_ltr[j]][0]
        ed_char_ltr = ctx_encode['offset_mapping'][j][ed_locs_ltr[j]][1]
        ans_pred_list_ltr.append(ctx[j][st_char_ltr: ed_char_ltr])

metric = np.array([one_to_many_metric(x, y, tokenizer) for x, y in zip(ans_pred_list_ltr, ans_gt_list)])
em = metric[:, 0].mean()
f1 = metric[:, 1].mean()

print(f'EM = {em}')
print(f'F1 = {f1}')

assert len(ans_pred_list_ltr) == 1000
json.dump(ans_pred_list_ltr, open("retrieve_and_read.json", 'w'))

Number of evaluation examples: 1000
EM = 0.526
F1 = 0.6016567861534152


A correct implementation of BM25 and the reader model should achieve an exact match of 50% or greater. Please download and submit the `retrieve_and_read.json` file generated by the code above.